In [6]:
# Import
import requests
import json
import pandas as pd
from google_api_key import google_key
from geojson import Feature, FeatureCollection, Point

In [2]:
# Import the clean json data
path = "../data/clean_data.json"

df = pd.read_json(path)
df.head()

,title,company_name,location,via,description,extensions,schedule_type,work_from_home,salary_hourly,salary_yearly,salary_standardized,description_tokens
0,"Data analyst | Wichita, KS | Fulltime",Dice,"Wichita, KS",via LinkedIn,"Role: Data analyst Location: Wichita, KS (Day ...","['12 hours ago', 'Full-time', 'No degree menti...",Full-time,NaN,NaN,NaN,NaN,"['excel', 'tableau', 'vba', 'power_bi']"
1,Quality Measures Data Analyst (Remote),Talentify.io,Anywhere,via LinkedIn,Talentify helps candidates around the world to...,"['13 hours ago', 'Work from home', 'Full-time']",Full-time,1.0,NaN,NaN,NaN,"['visio', 'word', 'sql', 'tableau', 'powerpoin..."
2,Senior Data Analyst,Naviguard,United States,via My Stateline Jobs,Senior Data Analyst\nAs a Senior Data Analyst ...,"['8 hours ago', 'Full-time']",Full-time,NaN,NaN,NaN,NaN,['sql']
3,EDI Data Analyst/Business Systems Analyst III ...,Talentify.io,Anywhere,via LinkedIn,Talentify helps candidates around the world to...,"['13 hours ago', 'Work from home', 'Full-time']",Full-time,1.0,NaN,NaN,NaN,[]
4,Data Analyst that can read and find places onl...,Upwork,Anywhere,via Upwork,My company is evaluating hundreds of counties ...,"['8 hours ago', 'Work from home', 'Contractor'...",Contractor,1.0,NaN,NaN,NaN,"['spreadsheet', 'excel', 'go']"


In [3]:
#create new dataframe for the locations of the job postings and the count of appearances in the dataset
location_df = df['location'].value_counts(dropna=True).rename_axis('location').reset_index(name='count')

#filter out only locations with city and state provided for location mapping
clean_location_df = location_df[location_df['location'].str.contains(r'\w+,\s\w{2}')]
clean_location_df.reset_index(drop=True, inplace=True)
clean_location_df

,location,count
0,"Kansas City, MO",528
1,"Oklahoma City, OK",380
2,"Jefferson City, MO",350
3,"Columbia, MO",211
4,"Tulsa, OK",200
...,...,...
505,"Bixby, OK",1
506,"Catoosa, OK",1
507,"Steedman, MO",1
508,"Sedgwick, KS",1


In [7]:
cities = []
lats = []
lons = []

#iterate through df and store location names to cities list
for i in range(len(clean_location_df)):
    cities.append(clean_location_df.loc[i, "location"])

#iterate through cities list, pass in city to google api, store latitude and longitude to lists from api response
for city in cities:

    target_url = "https://maps.googleapis.com/maps/api/geocode/json?address=+" + city + "&key=" + google_key

    response = requests.get(target_url).json()
    geometry = response["results"][0]["geometry"]

    lats.append(geometry["location"]["lat"])
    lons.append(geometry["location"]["lng"])


In [9]:

#create dataframe with cities, lats and lons
df = pd.DataFrame(list(zip(cities, lats, lons)),columns =['location', 'latitude', 'longitude'])
#merge this with the previous df to add lats and lons
final_location_df = pd.merge(clean_location_df, df, on='location', how='inner')

#convert df to geojson
geojson = {"type": "FeatureCollection", "features": []}

for _, row in final_location_df.iterrows():
    feature = {"type": "Feature", "geometry": {"type": "Point", "coordinates": [row['longitude'], row['latitude']]}, "properties": {"location": row['location'], "count": row['count']}}
    geojson['features'].append(feature)

#export to json

with open('../data/map.geojson', 'w') as fp:
    json.dump(geojson, fp)   
